In [4]:
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import openslide

thispath = Path.cwd().resolve()

def downsample_image(slide, downsampling_factor, mode="numpy"):
    """Downsample an Openslide at a factor.

    Takes an OpenSlide SVS object and downsamples the original resolution
    (level 0) by the requested downsampling factor, using the most convenient
    image level. Returns an RGB numpy array or PIL image.

    Args:
        slide: An OpenSlide object.
        downsampling_factor: Power of 2 to downsample the slide.
        mode: String, either "numpy" or "PIL" to define the output type.

    Returns:
        img: An RGB numpy array or PIL image, depending on the mode,
            at the requested downsampling_factor.
        best_downsampling_level: The level determined by OpenSlide to perform the downsampling.
    """

    # Get the best level to quickly downsample the image
    # Add a pseudofactor of 0.1 to ensure getting the next best level
    # (i.e. if 16x is chosen, avoid getting 4x instead of 16x)
    best_downsampling_level = slide.get_best_level_for_downsample(downsampling_factor + 0.1)

    # Get the image at the requested scale
    svs_native_levelimg = slide.read_region((0, 0), best_downsampling_level, slide.level_dimensions[best_downsampling_level])
    target_size = tuple([int(x//downsampling_factor) for x in slide.dimensions])
    img = svs_native_levelimg.resize(target_size)

    # By default, return a numpy array as RGB, otherwise, return PIL image
    if mode == "numpy":
        # Remove the alpha channel
        img = np.array(img.convert("RGB"))

    return img, best_downsampling_level

datadir = Path("/mnt/nas4/datasets/ToReadme/ExaMode_Dataset1/AOEC")

wsi_id = "000032269300415945"
max_tiles_coordinates = (100, 78)

metadata_preds = pd.read_csv(thispath.parent / "data" / "Mask_PyHIST_v2" / "LungAOEC_List1"
                             / wsi_id / f"{wsi_id}_densely_filtered_metadata_v2.csv")
pred_row = metadata_preds["row"].values
print(pred_row)
print(len(pred_row))
pred_column = metadata_preds["column"].values
print(pred_column)
print(len(pred_column))
lista_multi = sorted([a*b for a,b in zip(pred_row, pred_column)])
print(lista_multi)
print(len(lista_multi))

svs_files = [i for i in datadir.rglob("*.svs") if wsi_id in str(i)]

file = svs_files[0]

slide = openslide.OpenSlide(str(file))
mpp = slide.properties["openslide.mpp-x"]
print(slide.properties)

# Get a downsampled numpy array for the image"
tilecrossed_img = downsample_image(slide, 32, mode="numpy")[0]

# Calculate patch size in the mask
tilecross_patchsize = int(np.ceil(256 * (4/32)))

# Draw the grid at the scaled patchsize
x_shift, y_shift = tilecross_patchsize, tilecross_patchsize
gcol = [255, 0, 0]
tilecrossed_img[:, ::y_shift, :] = gcol
tilecrossed_img[::x_shift, :, :] = gcol

# Convert numpy array to PIL image
tilecrossed_img = Image.fromarray(tilecrossed_img, mode="RGB")

# Create object to draw the crosses for each tile
draw = ImageDraw.Draw(tilecrossed_img)

grid_coord = max_tiles_coordinates

# Counters for iterating through the tile-crossed image tiles
tc_w = 0
tc_h = 0
row, col, i, z = 0, 0, 0, 0
one_predict_col = pred_column[z]
one_predict_row = pred_row[z]

# Evaluate tiles using the selector function
while row < grid_coord[1]:

    start_w = col * (tilecross_patchsize)
    start_h = row * (tilecross_patchsize)

    # If we reach the edge of the image, we only can draw until the edge pixel
    if (start_w + tilecross_patchsize) >= tilecrossed_img.size[0]:
        cl_w = tilecrossed_img.size[0] - start_w
    else:
        cl_w = tilecross_patchsize

    if (start_h + tilecross_patchsize) >= tilecrossed_img.size[1]:
        cl_h = tilecrossed_img.size[1] - start_h
    else:
        cl_h = tilecross_patchsize

    # Draw the cross only if the tile has to be kept

    if (col == one_predict_col) and (row == one_predict_row):

        # From top left to bottom right
        draw.line([(start_w, start_h), (start_w + cl_w, start_h + cl_h)], fill=(0, 0, 255), width=3)

        # From bottom left to top right
        draw.line([(start_w, start_h + cl_h), (start_w + cl_w, start_h)], fill=(0, 0, 255), width=3)

        z +=1
        if z < len(pred_column):
            one_predict_col = pred_column[z]
            one_predict_row = pred_row[z]


    # Jump to the next column tile
    col += 1

    # Increase counter for metadata
    i += 1 

    # If we reach the right edge of the image, jump to the next row
    if col == grid_coord[0]:
        col = 0
        row += 1
   
        tc_w = 0
        tc_h = tc_h + tilecross_patchsize + 1


tilecrossed_outpath = (f"{thispath}/output.png")
tilecrossed_img.save(tilecrossed_outpath)

[ 4  4  4 ... 73 74 74]
3962
[53 55 56 ... 49 48 49]
3962
[54, 63, 66, 69, 75, 78, 80, 80, 81, 84, 84, 85, 87, 90, 90, 90, 93, 96, 96, 98, 100, 100, 102, 104, 108, 108, 108, 111, 112, 112, 114, 115, 116, 119, 120, 120, 120, 124, 125, 126, 126, 128, 130, 132, 132, 133, 135, 135, 136, 136, 138, 140, 140, 140, 140, 143, 144, 144, 144, 144, 145, 147, 148, 150, 150, 150, 152, 152, 154, 155, 156, 156, 156, 160, 160, 160, 161, 162, 162, 165, 165, 168, 168, 168, 170, 170, 171, 174, 175, 175, 176, 180, 180, 180, 180, 182, 184, 186, 189, 190, 192, 192, 192, 192, 195, 195, 196, 198, 198, 200, 200, 203, 204, 204, 204, 205, 205, 207, 208, 208, 210, 210, 210, 210, 210, 212, 215, 215, 216, 216, 216, 216, 216, 216, 217, 217, 220, 222, 222, 224, 224, 224, 224, 225, 225, 225, 228, 228, 231, 231, 231, 232, 232, 234, 234, 234, 235, 236, 238, 238, 238, 240, 240, 240, 240, 240, 240, 240, 240, 243, 244, 245, 245, 246, 246, 248, 248, 248, 250, 252, 252, 252, 252, 252, 252, 252, 252, 253, 255, 256, 256, 258, 2